In [1]:
import pandas as pd
import numpy as np

In [2]:
# # Setting up the device for GPU usage

from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
print(device)

cuda


In [3]:
%cd ../../

d:\python\Toxic-comment-classification


In [4]:
path = 'kaggle/input/'
submission_path = 'kaggle/working/transformers/'
comp = 'jigsaw-toxic-comment-classification-challenge/'
clean_data_path = 'clean_data/'
TRAIN_DATA_FILE=f'{path}{comp}train.csv.zip'
TEST_DATA_FILE=f'{path}{comp}test.csv.zip'
CLEAN_TRAIN_DATA_FILE=f'{clean_data_path}data_train_cleaned_vanilla2.txt'
CLEAN_TEST_DATA_FILE=f'{clean_data_path}data_test_cleaned_vanilla2.txt'
SAMPLE_SUBMISSION=f'{path}{comp}sample_submission.csv.zip'
checkpoint_path = 'model_checkpoint/transformers/'
classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]

In [5]:
def read_from_file(filename):
    with open(filename, 'r') as f:
        return f.read().splitlines()  

In [6]:
X_te = read_from_file(CLEAN_TEST_DATA_FILE)
X_te[:2]

['yo bitch ja rule is more succesful then you will ever be whats up with you and hating you sad should bitch slap ur pethedic white faces and get you to kiss my ass you guys sicken me ja rule is about pride in da music man dont diss that shit on him and nothin is wrong bein like tupac he was a brother white boys get things right next time',
 'from rfc the title is fine as it is imo']

In [7]:
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification
import torch

tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = DistilBertForSequenceClassification.from_pretrained('results\\checkpoint-17952')

d:\anaconda\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

In [13]:
import torch
from torch.utils.data import DataLoader, TensorDataset
import numpy as np

# Assuming tokenizer and device are already defined

def get_preds(test_data, model, batch_size=64):
    # Tokenize the entire dataset first
    test_inputs = tokenizer(test_data, truncation=True, padding="max_length", max_length=128, return_tensors='pt')
    dataset = TensorDataset(test_inputs['input_ids'], test_inputs['attention_mask'])
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False, num_workers=4)

    preds = []
    sigmoid = torch.nn.Sigmoid()

    model.eval()
    model.to(device)

    with torch.no_grad():
        for batch in dataloader:
            batch = {key: val.to(device) for key, val in zip(['input_ids', 'attention_mask'], batch)}
            outputs = model(**batch)
            probs = sigmoid(outputs.logits.cpu())
            preds.extend(probs.numpy())

    return np.array(preds)

preds = get_preds(X_te, model)

In [14]:
submission = pd.read_csv(SAMPLE_SUBMISSION)
submission[classes] = preds
submission.to_csv(f'{submission_path}distilbert-fine-tune.csv', index=False)